# Librerias

In [2]:
random_state = 43992294
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import category_encoders as ce
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.impute import SimpleImputer
import seaborn as sns
from scipy.stats import chi2_contingency
import scipy.stats as ss
import os
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MultiLabelBinarizer

from xgboost import XGBClassifier
from sklearn.preprocessing import FunctionTransformer

import ast

import warnings

from tqdm.auto import tqdm

from sklearn.feature_selection import RFECV

from sklearn.model_selection import StratifiedKFold, cross_val_score

from itertools import chain

import dask.dataframe as dd

from sklearn.utils import resample

from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier

# Cython imports
from tools import agrupar_categorias_cython, custom_one_hot_encoder_cython, boolean_features_ohe_cython, agrupar_edades_cython, expand_action_list_0_cython

# Fuciones

In [3]:
warnings.filterwarnings('ignore')
tqdm.pandas()

# Agrega filas que contienen categorías desconocidas o raras (poca frecuencia) al conjunto de entrenamiento
def augment_train_data(main_train_df, supplementary_df, umbral_raras=100):
    """
    Agrega filas del dataset suplementario al conjunto de entrenamiento principal
    basándose en categorías desconocidas y raras, evitando la duplicación de filas.

    Parámetros:
    - main_train_df (pd.DataFrame): DataFrame principal de entrenamiento.
    - supplementary_df (pd.DataFrame): DataFrame suplementario del cual se extraerán las filas.
    - umbral_raras (int): Umbral de frecuencia para considerar una categoría como rara.

    Retorna:
    - main_train_df (pd.DataFrame): DataFrame de entrenamiento actualizado.
    - categorias_desconocidas (dict): Diccionario actualizado de categorías desconocidas.
    - categorias_raras (dict): Diccionario actualizado de categorías raras.
    """
    # Definir columnas que no deseas tratar como categóricas
    columns_to_exclude = ['auction_list_0', 'action_list_1', 'action_list_2', 'auction_time']

    # Identificar columnas categóricas excluyendo las especificadas
    categorical_features = main_train_df.select_dtypes(include=['object']).columns.tolist()
    categorical_features = [col for col in categorical_features if col not in columns_to_exclude]
    
    # Crear un diccionario para almacenar las categorías desconocidas por columna
    categorias_desconocidas = {}
    
    # Iterar a través de cada columna categórica para identificar categorías desconocidas
    for columna in categorical_features:
        # Obtener las categorías únicas en el conjunto de entrenamiento
        categorias_train = set(main_train_df[columna].dropna().unique())
        
        # Obtener las categorías únicas en el dataset suplementario
        categorias_suplementario = set(supplementary_df[columna].dropna().unique())
        
        # Identificar las categorías en el dataset suplementario que no están en el entrenamiento
        desconocidas = categorias_suplementario - categorias_train
        
        # Almacenar las categorías desconocidas en el diccionario como una lista
        categorias_desconocidas[columna] = list(desconocidas)
    
    # Inicializar el diccionario para almacenar las categorías raras por columna
    categorias_raras = {}
    
    # Identificar categorías raras en el conjunto de entrenamiento
    for columna in categorical_features:
        # Contar la frecuencia de cada categoría
        frecuencia = main_train_df[columna].value_counts()
        
        # Identificar categorías que aparecen menos de umbral_raras veces
        raras = frecuencia[frecuencia < umbral_raras].index.tolist()
        
        # Almacenar en el diccionario
        categorias_raras[columna] = raras
    
    # Crear una máscara booleana para filas con categorías desconocidas o raras
    mask_desconocidas = pd.Series([False] * len(supplementary_df))
    mask_raras = pd.Series([False] * len(supplementary_df))
    
    for columna in categorical_features:
        # Actualizar la máscara para categorías desconocidas
        if categorias_desconocidas[columna]:
            mask_desconocidas = mask_desconocidas | supplementary_df[columna].isin(categorias_desconocidas[columna])
        
        # Actualizar la máscara para categorías raras
        if categorias_raras[columna]:
            mask_raras = mask_raras | supplementary_df[columna].isin(categorias_raras[columna])
    
    # Combinar ambas máscaras
    mask_total = mask_desconocidas | mask_raras
    
    # Filtrar filas únicas a agregar
    filas_a_agregar = supplementary_df[mask_total].drop_duplicates()
    
    # Mostrar información de agregación
    total_agregadas = len(filas_a_agregar)
    print(f"\nAgregando {total_agregadas} filas del dataset suplementario basadas en categorías desconocidas o raras.")
    
    # Agregar las filas al conjunto de entrenamiento
    main_train_df = pd.concat([main_train_df, filas_a_agregar], ignore_index=True)
    
    # Actualizar los diccionarios eliminando las categorías que ya han sido agregadas
    for columna in categorical_features:
        # Actualizar categorías desconocidas
        if categorias_desconocidas[columna]:
            categorias_agregadas = filas_a_agregar[columna].unique().tolist()
            categorias_desconocidas[columna] = [cat for cat in categorias_desconocidas[columna] if cat not in categorias_agregadas]
        
        # Actualizar categorías raras
        if categorias_raras[columna]:
            # Recontar la frecuencia después de agregar
            frecuencia = main_train_df[columna].value_counts()
            nuevas_raras = frecuencia[frecuencia < umbral_raras].index.tolist()
            categorias_raras[columna] = nuevas_raras
    
    return main_train_df, categorias_desconocidas, categorias_raras

# Agrega como columnas binarias las listas de la columna 'auction_list_0', 'action_list_1' y 'action_list_2'
def expand_list_dummies_cython(df, column, delimiter='|', prefix=None, suffix=None):
    """
    Expande una columna que contiene listas en múltiples columnas binarias usando un one-hot encoder optimizado con Cython.
    
    Parameters:
    - df (pd.DataFrame): DataFrame de pandas.
    - column (str): Nombre de la columna a expandir.
    - delimiter (str): Delimitador a usar en get_dummies (por defecto '|').
    - prefix (str, optional): Prefijo para las nuevas columnas binarias.
    - suffix (str, optional): Sufijo para las nuevas columnas binarias.
    
    Returns:
    - df_expanded (pd.DataFrame): DataFrame con las nuevas columnas binarias añadidas y la columna original eliminada.
    """
    print(f"Comenzando la expansión de la columna: '{column}'")
    
    # Copiar el DataFrame para evitar modificar el original
    df = df.copy()
    
    # Reemplazar NaN por listas vacías
    print(f"Reemplazando NaN en la columna '{column}' por listas vacías.")
    df[column] = df[column].fillna('[]')
    
    # Definir la función de parsing con impresión de errores
    def parse_list(x):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                # Convertir todos los elementos a strings
                return [str(item) for item in parsed]
            else:
                # Si no es una lista, tratar como un solo elemento
                return [str(x)]
        except (ValueError, SyntaxError):
            # En caso de error al parsear, retornar una lista vacía
            return []
    
    # Aplicar la función de parsing con una barra de progreso
    df[column] = df[column].progress_apply(parse_list)
    
    # Convertir la columna en una lista de listas para pasarla a la función Cythonizada
    data_list = df[column].tolist()
    
    # Llamar a la función optimizada en Cython
    unique_categories, binary_matrix = custom_one_hot_encoder_cython(data_list)
    print(f"Codificación completada. {len(unique_categories)} categorías únicas encontradas.")
    
    # Crear un DataFrame binario usando la matriz devuelta por Cython
    binary_df = pd.DataFrame(binary_matrix, index=df.index, columns=unique_categories)

    if prefix:
        binary_df = binary_df.add_prefix(f"{prefix}_")
    if suffix:
        binary_df = binary_df.add_suffix(f"_{suffix}")
    
    # Concatenar las columnas binarias al DataFrame original con una barra de progreso
    print("Concatenando las columnas binarias al DataFrame original.")
    for col in tqdm(binary_df.columns, desc="Concatenando columnas binarias"):
        if col in df.columns:
            df[col] = np.where((df[col] == 1) | (binary_df[col] == 1), 1, 0)
        else:
            df[col] = binary_df[col]
    
    # Eliminar la columna original ya que ha sido expandida
    print(f"Eliminando la columna original '{column}' del DataFrame.")
    df = df.drop(columns=[column])
    
    print(f"Expansión de la columna '{column}' completada exitosamente.\n")
    
    return df

# One-hot encode de columnas booleanas utilizando Cython
def boolean_features_ohe(df, columns_to_encode=['auction_boolean_0', 'auction_boolean_1', 'auction_boolean_2']):
    """
    Realiza one-hot encoding en columnas booleanas especificadas utilizando una función optimizada con Cython.
    Además, muestra el progreso del procesamiento utilizando tqdm y añade comentarios explicativos.

    Parámetros:
    - df (pd.DataFrame): DataFrame original que contiene las columnas booleanas a codificar.
    - columns_to_encode (list): Lista de nombres de columnas booleanas a codificar.

    Retorna:
    - df_expanded (pd.DataFrame): DataFrame con las nuevas columnas codificadas añadidas y las columnas booleanas originales eliminadas.
    """
    # Copiar el DataFrame para evitar modificar el original
    df = df.copy()
    print("Inicio del proceso de one-hot encoding para las columnas booleanas especificadas.")

    # Paso 1: Encontrar todos los valores únicos en las columnas a codificar
    unique_values_set = set()
    print("Recopilando valores únicos de las columnas a codificar:")
    for col in tqdm(columns_to_encode, desc="Procesando columnas para valores únicos"):
        unique_vals_col = df[col].dropna().unique()
        unique_values_set.update(unique_vals_col)
    unique_values = sorted(unique_values_set)
    print(f"Valores únicos encontrados: {unique_values}")

    # Paso 2: Convertir las columnas a listas de listas para ser procesadas en Cython
    list_data = []
    print("Convirtiendo las columnas booleanas a listas de listas para Cython:")
    for col in tqdm(columns_to_encode, desc="Convertir columnas a listas"):
        column_list = df[col].astype(str).tolist()  # Mantener los valores como strings
        list_data.append(column_list)
    print("Conversión completada.")

    # Paso 3: Procesar los datos con la función optimizada en Cython
    print("Realizando one-hot encoding utilizando la función optimizada en Cython:")
    ohe_result = boolean_features_ohe_cython(list_data, unique_values)
    print("One-hot encoding completado.")

    # Paso 4: Convertir el resultado de Cython a un DataFrame, alineando el índice con df
    print("Creando el DataFrame de columnas codificadas:")
    ohe_df = pd.DataFrame(ohe_result, columns=unique_values, index=df.index)
    print(f"DataFrame de one-hot encoding creado con {len(ohe_df.columns)} columnas y {ohe_df.shape[0]} filas.")

    # Paso 5: Concatenar las nuevas columnas codificadas al DataFrame original
    print("Concatenando las columnas codificadas al DataFrame original:")
    df_expanded = pd.concat([df, ohe_df], axis=1)
    print(f"Concatenación completada. El DataFrame ahora tiene {df_expanded.shape[1]} columnas y {df_expanded.shape[0]} filas.")

    # Paso 6: Eliminar las columnas booleanas originales del DataFrame
    print("Eliminando las columnas booleanas originales del DataFrame:")
    df_expanded.drop(columns=columns_to_encode, inplace=True)
    print(f"Columnas eliminadas: {columns_to_encode}")

    print("Proceso de one-hot encoding finalizado exitosamente.\n")

    return df_expanded

# Extensión de características temporales (día de la semana, momento del día, etc.) y festividades
def time_features_extension(df):
    """
    Procesa las características temporales del DataFrame y agrega nuevas columnas derivadas relacionadas con el tiempo y festividades.
    """

    # Convertir 'auction_time' de timestamp a una fecha legible
    df['auction_time'] = pd.to_datetime(df['auction_time'], unit='s')

    # Reemplazar NaN en 'timezone_offset' por 0
    df['timezone_offset'] = df['timezone_offset'].fillna(0)

    # Ajustar la hora según el 'timezone_offset' para obtener la hora local
    df['auction_time_local'] = df.apply(
        lambda row: row['auction_time'] + pd.DateOffset(hours=row['timezone_offset']), axis=1
    )

    # Crear la columna 'week_day' (1 para lunes, 7 para domingo)
    df['week_day'] = df['auction_time_local'].dt.weekday + 1

    # Crear la columna 'moment_of_the_day' (1 para temprano, 2 para tarde, 3 para noche)
    df['moment_of_the_day'] = pd.cut(df['auction_time_local'].dt.hour, 
                                     bins=[0, 12, 18, 24], labels=[1, 2, 3], include_lowest=True, right=False)

    # Eliminar las columnas originales 'auction_time', 'timezone_offset' y 'auction_time_local'
    df.drop(columns=['auction_time', 'timezone_offset', 'auction_time_local'], inplace=True)

    return df

# Agrupación de edades en rangos numéricos
def age_group(df, columna_edad):
    """
    Agrupa las edades en rangos numéricos utilizando Cython para mejorar el rendimiento.

    Parámetros:
    - df (pd.DataFrame): DataFrame que contiene la columna de edades.
    - columna_edad (str): Nombre de la columna que contiene las edades.

    Retorna:
    - df (pd.DataFrame): DataFrame con la nueva columna 'age_group' que representa el rango de edad.
    """
    # Convertir la columna de edad a una lista
    edades = df[columna_edad].tolist()

    # Usar la función Cythonizada para agrupar las edades
    df['age_group'] = agrupar_edades_cython(edades)

    # Eliminar la columna original de edades
    df.drop(columns=[columna_edad], inplace=True)

    return df

# Agrupo action_list_0 a auction_list_0
def expand_action_list_0(df):
    """
    Expande la columna 'action_list_0' en valores únicos y marca con 1 las columnas existentes o las crea si es necesario.

    Parámetros:
    - df (pd.DataFrame): DataFrame que contiene la columna 'action_list_0' y otras columnas de listas ya expandidas.

    Retorna:
    - df (pd.DataFrame): DataFrame actualizado con las columnas de valores únicos de 'action_list_0'.
    """

    # Convertir la columna 'action_list_0' y las columnas existentes a listas
    action_list_0 = df['action_list_0'].tolist()
    existing_columns = df.columns.tolist()
    
    # Inicializar la matriz actual
    current_matrix = df.values.tolist()

    # Llamar a la función Cythonizada
    updated_matrix = expand_action_list_0_cython(action_list_0, existing_columns, current_matrix)

    # Convertir la matriz actualizada de vuelta a un DataFrame
    df_updated = pd.DataFrame(updated_matrix, columns=existing_columns)

    # Eliminar la columna 'action_list_0'
    df_updated.drop(columns=['action_list_0'], inplace=True)

    return df_updated

# Concateno las categorias de cada nivel
def create_level_combination(df):
    """
    Creates a new column 'level_combination' by concatenating the first three characters 
    of each 'action_categorical' level columns and removes the original level columns.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the columns:
      'action_categorical_0', 'action_categorical_1', 'action_categorical_2', 
      'action_categorical_3', 'action_categorical_4'.

    Returns:
    - pd.DataFrame: The DataFrame with the new 'level_combination' column and without the original level columns.
    """
    level_columns = [
        'action_categorical_0',
        'action_categorical_1',
        'action_categorical_2',
        'action_categorical_3',
        'action_categorical_4'
    ]
    df['level_combination'] = df[level_columns].astype(str).apply(
        lambda x: ''.join([s[:3] for s in x]), axis=1
    )
    df.drop(columns=level_columns, inplace=True)
    return df

# Heigh x Width a columna
def hxw_column(df):
    """
    Crea una nueva columna 'hxw' multiplicando 'creative_height' y 'creative_width'.
    Si alguno de los dos tiene un NaN, 'hxw' se establece en 0.
    Elimina las columnas originales 'creative_height' y 'creative_width'.
    
    Parámetros:
    - df (pd.DataFrame): DataFrame que contiene las columnas 'creative_height' y 'creative_width'.
    
    Retorna:
    - pd.DataFrame: DataFrame con la nueva columna 'hxw' añadida y las columnas originales eliminadas.
    """
    df['hxw'] = df['creative_height'] * df['creative_width']
    df.loc[df['creative_height'].isna() | df['creative_width'].isna(), 'hxw'] = 0
    df.drop(columns=['creative_height', 'creative_width'], inplace=True)
    return df

# Gender to number
def encode_gender(df):
    """
    Reemplaza los valores de la columna 'gender' de la siguiente manera:
    'f' -> 1, 'm' -> 2, 'o' -> 0 y NaN -> -1.
    
    Parámetros:
    - df (pd.DataFrame): DataFrame que contiene la columna 'gender'.
    
    Retorna:
    - pd.DataFrame: DataFrame con la columna 'gender' codificada.
    """
    df['gender'] = df['gender'].map({'f': 1, 'm': 2, 'o': 0}).fillna(-1).astype(int)
    return df

# creative_categorical_11, creative_categorical_9 y creative_categorical_10 a dos columnas
def creatives2unique(df):
    """
    Crea o actualiza dos columnas en el DataFrame, una para cada valor único en las columnas
    'creative_categorical_11', 'creative_categorical_10', y 'creative_categorical_9'.
    Si las columnas ya existen, actualiza los valores a 1 donde ese valor aparece en alguna
    de las tres columnas en esa fila.

    Parámetros:
    - df (pd.DataFrame): DataFrame que contiene las columnas 'creative_categorical_11',
                         'creative_categorical_10', y 'creative_categorical_9'.

    Retorna:
    - pd.DataFrame: DataFrame con las nuevas columnas añadidas o actualizadas.
    """
    unique_values = {'65dcab89', '43c867fd'}
    columns_to_check = [
        'creative_categorical_11',
        'creative_categorical_10',
        'creative_categorical_9'
    ]

    for val in unique_values:
        if val in df.columns:
            # Si la columna ya existe, actualizamos los valores a 1 donde corresponde
            df[val] = df[val] | df[columns_to_check].eq(val).any(axis=1).astype(int)
        else:
            # Si no existe, creamos la columna con 1 donde corresponde
            df[val] = df[columns_to_check].eq(val).any(axis=1).astype(int)

    # Eliminar las columnas originales
    df.drop(columns=columns_to_check, inplace=True)
    
    return df

# Juntar todas las variables categoricas y hacer OHE
def process_combineta(df):
    """
    Procesa las columnas proporcionadas en combineta, creando un set con valores únicos,
    y generando columnas binarias para cada uno de esos valores. Si la columna ya existe,
    actualiza las filas con un 1 donde corresponda. Luego, elimina las columnas originales.

    Parámetros:
    - df (pd.DataFrame): DataFrame original.
    - combineta_columns (list): Lista de columnas a procesar.

    Retorna:
    - df (pd.DataFrame): DataFrame con las columnas binarias añadidas y las columnas originales eliminadas.
    """
    combineta_columns = ['creative_categorical_0', 'creative_categorical_5', 'auction_categorical_0', 'auction_categorical_1', 'auction_categorical_11', 'auction_categorical_7', 'auction_categorical_8', 'auction_categorical_9', 'action_categorical_6', 'action_categorical_7', 'auction_categorical_3', 'auction_categorical_4', 'auction_categorical_5', 'auction_categorical_6', 'auction_categorical_10', 'auction_categorical_12', 'creative_categorical_1', 'creative_categorical_12', 'creative_categorical_2', 'creative_categorical_3', 'creative_categorical_4', 'creative_categorical_6', 'creative_categorical_7', 'creative_categorical_8', 'device_id', 'device_id_type', 'level_combination']

    # Unir todas las columnas de combineta en una sola columna de listas
    df['combined_combineta'] = df[combineta_columns].astype(str).agg(
        lambda x: '[' + ', '.join([f"'{str(item).strip()}'" for item in x if item != 'nan']) + ']', axis=1)

    # Usar la función expand_list_dummies_cython para descomponer la lista y crear las columnas binarias
    df = expand_list_dummies_cython(df, 'combined_combineta')
    
    df.drop(columns=combineta_columns, inplace=True, errors='raise')
    
    return df

# Procesamiento optimizado de un DataFrame
def process_optimized(df):
    """
    Aplica una serie de transformaciones al DataFrame utilizando una función Cython optimizada.
    
    Parámetros:
    - df (pd.DataFrame): DataFrame a procesar.
    
    Retorna:
    - df (pd.DataFrame): DataFrame procesado.
    """
    # Definir el número total de pasos para la barra de progreso
    total_steps = 14
    
    # Inicializar la barra de progreso
    with tqdm(total=total_steps, desc="Procesando DataFrame", unit="paso") as pbar:
        
        print("Comenzando el procesamiento optimizado del DataFrame.")
        print("Eliminando columnas innecesarias.")
        df = df.drop('action_categorical_5', axis=1)
        df = df.drop('auction_categorical_2', axis=1)
        pbar.update(1)

        print("Expansión de columnas temporales")
        df = time_features_extension(df)
        pbar.update(1)

        df = age_group(df, 'auction_age')
        pbar.update(1)

        print("Agrupando columnas de nivel")
        df = create_level_combination(df)
        pbar.update(1)

        print("Modificando columnas de genero")
        df = encode_gender(df)
        pbar.update(1)
        
        print("Modificando columna de video")
        df['has_video'] = df['has_video'].apply(lambda x: 1 if x == True else 0)
        pbar.update(1)

        print("Juntando medidas")
        df = hxw_column(df)
        pbar.update(1)

        print("Expansión de columnas booleanas.")
        df = boolean_features_ohe(df)
        pbar.update(1)

        print("Creando columnas de creatividad")
        df = creatives2unique(df)
        pbar.update(1)
        
        columns_to_expand = ['auction_list_0', 'action_list_1']

        # Bucle para realizar las operaciones
        for col in columns_to_expand:
            print(f"Expansión de columnas de listas para {col}.")
            
            # Crear la variable 'idx_position' con la última columna antes de la expansión
            idx_position = df.columns.get_loc(df.columns[-1])
            
            # Expansión de la columna
            df = expand_list_dummies_cython(df, col)
            pbar.update(1)

            if col == 'action_list_1':
                df = expand_list_dummies_cython(df,'action_list_2')
                pbar.update(1)
            
            if col == 'auction_list_0':
                # Bucle para recorrer las columnas que empiezan con 'AND' o 'APL'
                if 'AND-APL' not in df.columns:
                    df['AND-APL'] = 0  # Inicializar la columna 'AND-APL'

                for column in df.columns:
                    if column.startswith('AND') or column.startswith('APL'):
                        # Poner un 1 en 'AND-APL' si la columna actual tiene un 1 en esa fila
                        df['AND-APL'] = df['AND-APL'] | df[column]

                # Eliminar todas las columnas que empiezan con 'AND' o 'APL' excepto la columna 'AND-APL'
                columns_to_drop = [column for column in df.columns if (column.startswith('AND') or column.startswith('APL')) and column != 'AND-APL']
                df.drop(columns=columns_to_drop, inplace=True)

                print("Complementamos con la columna 'action_list_0'")
                df = expand_action_list_0(df)
                pbar.update(1)

            # Crear la lista de columnas numéricas a partir de la siguiente columna después de 'idx'
            categorical_num = df.iloc[:, idx_position:].select_dtypes(include=['number']).columns

            # Eliminar columnas numéricas con menos de 1000 valores iguales a 1
            for column in categorical_num:
                if (df[column] == 1).sum() < 1000 and column != 'AND-APL':
                    df.drop(column, axis=1, inplace=True)
    
        
        print("Agrupando categorias poco frecuentes")

        categorical_str = df.select_dtypes(include=['object']).columns
        categorical_str = categorical_str[categorical_str != 'device_id']
        
        categorical_num = df.select_dtypes(include=['number']).columns
        
        # Convertir a matriz bidimensional
        data_matrix = df[categorical_str].values.tolist()
        data_matrix_cython = [list(row) for row in data_matrix]
        
        # Llamar a la función Cythonizada
        df_cython_data = agrupar_categorias_cython(
            categorical_features=categorical_str.tolist(),
            data=data_matrix_cython,
            umbral=1000  # Umbral de frecuencia para considerar una categoría como rara
        )

        # Reasignar los datos al DataFrame
        for i, col in enumerate(categorical_str):
            df[col] = [row[i] for row in df_cython_data]

        idx_position = df.columns.get_loc(df.columns[-1])
        
        df = process_combineta(df)

        # Crear la lista de columnas numéricas a partir de la siguiente columna después de 'idx'
        categorical_num = df.iloc[:, idx_position:].select_dtypes(include=['number']).columns

        # Eliminar columnas numéricas con menos de 1000 valores iguales a 1
        for column in categorical_num:
            if (df[column] == 1).sum() < 1000:
                df.drop(column, axis=1, inplace=True)

        pbar.update(1)

        
    return df

# Función para ajustar el tipo de datos de una columna para que Dask tome Nan como valor válido
def adjust_dtype(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'Int64'
    elif pd.api.types.is_float_dtype(dtype):
        return 'float64'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'boolean'
    else:
        return 'object'
    
# Procesamiento de datos con Dask
def process_data_with_dask(df, npartitions=10, meta_df=None):
    """
    Procesa un DataFrame utilizando Dask para distribuir el trabajo en varias particiones.
    Aplica la función process_optimized a cada partición del DataFrame.

    Parámetros:
    - df (pd.DataFrame): El DataFrame de pandas a procesar.
    - npartitions (int): Número de particiones en las que se dividirá el DataFrame para su procesamiento.

    Retorna:
    - final_df (pd.DataFrame): El DataFrame procesado y concatenado.
    """
    # Convertir el DataFrame de pandas a Dask con el número de particiones especificado
    dask_df = dd.from_pandas(df, npartitions=npartitions)
    
    # Crear el meta DataFrame con tipos ajustados
    meta = df.head(0).copy()
    for col in meta.columns:
        meta[col] = meta[col].astype(adjust_dtype(df[col].dtype))
        
    # Aplicar la función con map_partitions y especificar el meta
    dask_df = dask_df.map_partitions(
        lambda df_partition: process_optimized(df_partition),
        meta=meta
    )

    # Ejecutar el cálculo distribuido y convertir el resultado a pandas
    final_df = dask_df.compute()

    return final_df

# Función para calcular la estadística de Cramér's V
def cramers_v(confusion_matrix):
    """
    Calcula la estadística de Cramér's V para medir la asociación entre dos variables categóricas.

    Parameters:
    - confusion_matrix: Matriz de confusión (tabla de contingencia) entre dos variables.

    Returns:
    - Cramér's V: Valor entre 0 y 1 que indica la fuerza de la asociación.
    """
    # Calcular el estadístico chi-cuadrado
    chi2 = chi2_contingency(confusion_matrix, correction=False)[0]
    # Número total de observaciones
    n = confusion_matrix.sum().sum()
    # Obtener el número de filas y columnas de la matriz de confusión
    r, k = confusion_matrix.shape
    if min(r, k) == 1:
        return np.nan  # Evitar dividir por cero
    # Calcular Cramér's V
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

def handle_none(value):
    return 'None' if value is None else str(value)

# Limpieza de datos

## Concatenamos datasets

In [ ]:
train_data_21 = pd.read_csv('data/ctr_21.csv')

In [13]:
train_data_20 = pd.read_csv('data/ctr_20.csv')
train_data_19 = pd.read_csv('data/ctr_19.csv')
train_data_18 = pd.read_csv('data/ctr_18.csv')
train_data_17 = pd.read_csv('data/ctr_17.csv')
train_data_16 = pd.read_csv('data/ctr_16.csv')
train_data_15 = pd.read_csv('data/ctr_15.csv')

In [ ]:
supplementary_datasets = [
    ('1', train_data_20),
    ('2', train_data_19),
    ('3', train_data_18),
    ('4', train_data_17),
    ('5', train_data_16),
    ('6', train_data_15)
]

train_data = train_data_21

In [ ]:
for nombre, dataset in supplementary_datasets:
    print(f"\nProcesando dataset {nombre}/{len(supplementary_datasets)}")
    train_data, categorias_desconocidas, categorias_raras = augment_train_data(train_data, dataset)

In [ ]:
print(train_data.shape[0])

print(train_data_20.shape[0] + train_data_19.shape[0] + train_data_18.shape[0] + train_data_17.shape[0] + train_data_16.shape[0] + train_data_15.shape[0] + train_data_21.shape[0])

In [ ]:
train_data.to_csv('train_data_combined.csv', index=False)

## Ingenieria de atributos

In [ ]:
train_data = pd.read_csv('train_data_combined.csv')

In [13]:
test = pd.read_csv('data/ctr_test.csv')

In [6]:
# Categorias numericas
numeric_columns = train_data.select_dtypes(include=['number']).columns.tolist()

# Categorias categóricas
categorical_features = train_data.select_dtypes(include=['object']).columns.tolist()

#### Distribucion de 'Label' 

In [ ]:
# Imprimir la cantidad de filas del dataset combinado
print(f"Cantidad de filas en el dataset combinado: {train_data.shape[0]}")
print(f"Cantidad de columnas en el dataset combinado: {train_data.shape[1]}")

# Ver porcentaje de clics vs no clics en la columna Label
label_counts = train_data['Label'].value_counts(normalize=True) * 100
print("\nPorcentaje de clics (1) y no clics (0):")
print(label_counts)

# Cantidad de clics (1) y no clics (0)
label_counts_abs = train_data['Label'].value_counts()
print("\nCantidad de clics (1) y no clics (0):")
print(label_counts_abs)

#### Correlaciones con 'Label'

##### Numeric features

In [ ]:
# Correlation analysis with 'Label'
numeric_data = train_data[numeric_columns]
correlation_with_label = numeric_data.corr()['Label'].sort_values(key=abs, ascending=False)

# Print correlations with 'Label'
print("Correlation of features with 'Label' (sorted by absolute value):")
for feature, corr in correlation_with_label.items():
    if feature != 'Label':
        print(f"{feature}: {corr:.4f}")

# Visualize top correlations with 'Label'
plt.figure(figsize=(12, 8))
top_correlations = correlation_with_label.drop('Label').abs()
sns.barplot(x=top_correlations.values, y=top_correlations.index)
plt.title('Numerical Features Correlated with Label')
plt.xlabel('Absolute Correlation')
plt.tight_layout()
plt.show()

##### Categorical features

In [ ]:
# Calculate Cramer's V for each categorical feature with 'Label'
cramer_v_results = {}
for col in categorical_features:
    confusion_matrix = pd.crosstab(train_data[col], train_data['Label'])
    cramer_v_results[col] = cramers_v(confusion_matrix)

# Sort results
cramer_v_results = dict(sorted(cramer_v_results.items(), key=lambda item: item[1], reverse=True))

# Visualize top Cramer's V results
plt.figure(figsize=(12, 8))
top_features = dict(list(cramer_v_results.items())[:20])
sns.barplot(x=list(top_features.values()), y=list(top_features.keys()))
plt.title("Top 20 Categorical Features by Cramer's V with Label")
plt.xlabel("Cramer's V")
plt.tight_layout()
plt.show()

# Print results
print("Cramer's V for categorical features with 'Label':")
for feature, v in top_features.items():
    print(f"{feature}: {v:.4f}")

#### Missing values

In [ ]:
# Análisis de datos faltantes (sin cambios)
missing_data = train_data.isnull().sum() / len(train_data) * 100
missing_data = missing_data[missing_data > 10].sort_values(ascending=False)  # Filtrar las columnas con más del 10% de datos faltantes

# Visualizar datos faltantes
plt.figure(figsize=(10, 6))
sns.barplot(x=missing_data.values, y=missing_data.index)
plt.title('Percentage of Missing Data by Numerical Feature (>10%)')
plt.xlabel('Percentage Missing')
plt.tight_layout()
plt.show()

# Imprimir resultados
print("\nPercentage of Missing Data by Feature (>10% missing):")
for feature, percentage in missing_data.items():
    print(f"{feature}: {percentage:.2f}%")

In [ ]:
# Crear un DataFrame de ejemplo
data = {
    'auction_boolean_0': ['47980dda', '43c867fd', None, '79ceee49', '47980dda', 'unknown'],
    'auction_boolean_1': ['79ceee49', None, '79ceee49', None, '1', '0'],
    'auction_boolean_2': ['65dcab89', None, '43c867fd', '65dcab89', '65dcab89', '43c867fd'],
    'Label': [0, 1, 0, 1, 0, 1]
}

# Crear el DataFrame
train_data_combined = pd.DataFrame(data)

# Seleccionar solo las columnas relevantes
df = train_data_combined[['auction_boolean_0', 'auction_boolean_1', 'auction_boolean_2', 'Label']]

print("DataFrame Original:")
print(df)
print("\n")

# Convertir el DataFrame a una lista de listas
data_list = df.values.tolist()

# Definir columnas categóricas y columnas a excluir
categorical_features = ['auction_boolean_0', 'auction_boolean_1', 'auction_boolean_2']
columns_to_exclude = []  # No se excluye ninguna en este ejemplo

# Aplicar la función de agrupación de categorías en Cython
data_processed = agrupar_categorias_cython(categorical_features, data_list, umbral=2)

# Convertir la lista de listas de nuevo a DataFrame para visualizar
df_processed = pd.DataFrame(data_processed, columns=['auction_boolean_0', 'auction_boolean_1', 'auction_boolean_2', 'Label'])

print("DataFrame después de Agrupar Categorías con Cython:")
print(df_processed)

#### Valores unicos y su frecuencia

In [ ]:
for column in train_data.columns:
    if train_data[column].value_counts() < 1000:
        print(f"Columna: {column}")
        print(f"Valores únicos y su frecuencia:")
        print(train_data[column].value_counts())
        print("-" * 50)  # Separador entre columnas

In [ ]:
# Diccionario para almacenar los valores únicos de cada columna
unique_values = {col: set(train_data[col].dropna().unique()) for col in categorical_features}

# Diccionario para almacenar las columnas que tienen valores comunes
common_columns = {}

# Comparar las columnas entre sí para ver qué valores comparten
for i in range(len(categorical_features)):
    for j in range(i + 1, len(categorical_features)):
        col1 = categorical_features[i]
        col2 = categorical_features[j]
        
        # Ver los valores que se repiten entre las dos columnas
        common_values = unique_values[col1].intersection(unique_values[col2])
        
        if common_values:
            # Almacenar las columnas con valores comunes
            if (col1, col2) not in common_columns:
                common_columns[(col1, col2)] = common_values

# Ver las columnas que tienen valores comunes y sus unique values
for col_pair, common_vals in common_columns.items():
    col1, col2 = col_pair
    print(f"\nValores comunes entre {col1} y {col2}: {common_vals}")
    print(f"Valores únicos de {col1}: {len(unique_values[col1])}")
    print(f"Valores únicos de {col2}: {len(unique_values[col2])}")

##### action_categorical_5 (motivos para eliminarlo)

In [ ]:
value_to_check = '6bc0e29c'
filtered_rows = test[test['action_categorical_5'] == value_to_check]

# Verificar si el valor aparece en otras columnas categóricas
for col in categorical_features:
    if col != 'action_categorical_5':  # Evitar verificar la columna original
        matching_rows = filtered_rows[filtered_rows[col] == value_to_check]
        if not matching_rows.empty:
            print(f"El valor '{value_to_check}' también aparece en la columna '{col}'")

# Ver filas donde el valor aparece solo en action_categorical_5
only_in_action_5 = filtered_rows[~filtered_rows[categorical_features].isin([value_to_check]).any(axis=1)]
print(f"Filas donde el valor '{value_to_check}' solo aparece en 'action_categorical_5':")
print(only_in_action_5)

In [ ]:
value_to_check = '79ceee49'
filtered_rows = test[test['action_categorical_5'] == value_to_check]

# Verificar si el valor aparece en otras columnas categóricas
for col in categorical_features:
    if col != 'action_categorical_5':  # Evitar verificar la columna original
        matching_rows = filtered_rows[filtered_rows[col] == value_to_check]
        if not matching_rows.empty:
            print(f"El valor '{value_to_check}' también aparece en la columna '{col}'")

# Ver filas donde el valor aparece solo en action_categorical_5
only_in_action_5 = filtered_rows[~filtered_rows[categorical_features].isin([value_to_check]).any(axis=1)]
print(f"Filas donde el valor '{value_to_check}' solo aparece en 'action_categorical_5':")
print(only_in_action_5)

##### auction_categorical_2 (motivos para eliminarlo)

In [ ]:
# Show unique values of auction_categorical_2
print("Unique values of 'auction_categorical_2':")
print(train_data['auction_categorical_2'].unique())

#### Levels features

In [ ]:
level_features = ['action_categorical_0', 'action_categorical_1', 'action_categorical_2', 'action_categorical_3', 'action_categorical_4']

print(f"Valores únicos y su frecuencia:")
for column in train_data.columns:
    if column in level_features:
        print(f"Columna: {column}")
        print(train_data[column].value_counts())
        print("-" * 50)  # Separador entre columnas

In [ ]:
# Bucle para contar los NaNs en cada columna
for column in level_features:
    num_nans = train_data[column].isna().sum()
    print(f"Columna {column} tiene {num_nans} valores NaN.")

In [ ]:
# Crear una columna con las combinaciones únicas de las 5 columnas pero tomando solo las dos primeras letras de cada valor
truncated_cols = train_data[level_features].applymap(lambda x: x[:3])

# Concatenar las columnas truncadas
train_data['combination'] = truncated_cols.apply(lambda row: ''.join(row.values), axis=1)

# Obtener las combinaciones únicas y sus frecuencias
combination_counts = train_data['combination'].value_counts().reset_index()
combination_counts.columns = ['Combination', 'Frequency']

# Mostrar las primeras 10 combinaciones más comunes
print(combination_counts.head(25))
print(f"De: {len(combination_counts)} combinaciones únicas.")

# Graficar las combinaciones más frecuentes
plt.figure(figsize=(12, 6))
top_combinations = combination_counts.head(25)  # Mostrar las 25 combinaciones más comunes
sns.barplot(x='Frequency', y='Combination', data=top_combinations)
plt.title("Top 25 Combinations of Action Categorical Columns out of {} Unique Combinations".format(len(combination_counts)))
plt.xlabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
# Combinaciones menos comunes (< 10000 veces)
print(combination_counts[combination_counts['Frequency'] <= 1])

# Suma de todas las frecuencias de las combinaciones menos comunes
print(f"Suma de las frecuencias de las combinaciones menos comunes: {combination_counts[combination_counts['Frequency'] <= 1]['Frequency'].sum()}")


In [ ]:
levels_df = create_level_combination(train_data)

levels_df['level_combination']

#### Boolean Features

In [ ]:
boolean_features = ['auction_boolean_0', 'auction_boolean_1', 'auction_boolean_2']

for column in train_data.columns:
    if column in boolean_features:
        print(f"Columna: {column}")
        print(f"Valores únicos y su frecuencia:")
        print(train_data[column].value_counts())
        print("-" * 50)

In [ ]:
# Bucle para contar los NaNs en cada columna
for column in boolean_features:
    num_nans = train_data[column].isna().sum()
    print(f"Columna {column} tiene {num_nans} valores NaN.")

Creamos una funcion para resolverl el problema de los valores booleanos

In [ ]:
# Crear un DataFrame de ejemplo
data = {
    'auction_boolean_0': ['47980dda', '43c867fd', None, '79ceee49'],
    'auction_boolean_1': ['79ceee49', None, '79ceee49', None],
    'auction_boolean_2': ['65dcab89', None, '43c867fd', '65dcab89'],
    'Label': [0, 1, 0, 1]
}

# Crear el DataFrame
train_data = pd.DataFrame(data)

# Imprimir el DataFrame original
print("DataFrame Original:")
print(train_data)

# Aplicar la función personalizada de one-hot encoding
train_data_encoded = boolean_features_ohe(train_data)

# Imprimir el DataFrame resultante
print("\nDataFrame después del One-Hot Encoding personalizado:")
print(train_data_encoded)

#### Time features

In [ ]:
time_features = ['auction_time', 'auction_age', 'timezone_offset']

for column in train_data.columns:
    if column in time_features:
        print(f"Columna: {column}")
        print(f"Valores únicos y su frecuencia:")
        print(train_data[column].value_counts())
        print("-" * 50)

In [ ]:
# Bucle para contar los NaNs en cada columna
for column in time_features:
    num_nans = train_data[column].isna().sum()
    print(f"Columna {column} tiene {num_nans} valores NaN.")

In [ ]:
# Asegúrate de tener la columna 'auction_age' en tu train
if 'auction_age' in train_data.columns:
    # Obtener los valores únicos y su frecuencia
    unique_ages = train_data['auction_age'].value_counts().sort_index()

    # Imprimir cada edad y su frecuencia
    for age, frequency in unique_ages.items():
        print(f"Edad: {age}, Frecuencia: {frequency}")


##### Funcion para crear atributos temporales

In [ ]:
# time_features_extension

data = {
    'auction_time': [
        1545676800,  # 24 de Diciembre de 2018, 22:00:00
        1483228800,  # 31 de Diciembre de 2016, 23:00:00
        1288396800,  # 30 de Octubre de 2010, 10:00:00
        1412899200,  # 10 de Octubre de 2014, 15:00:00
        1483574400   # 5 de Enero de 2017, 09:00:00
    ],
    'timezone_offset': [1, -2, 3, 0, 5]  # Diferentes zonas horarias
}

df = pd.DataFrame(data)

df = time_features_extension(df)

# Imprimir el DataFrame resultante
print("\nDataFrame después de procesar el tiempo de subasta:")
print(df)

In [ ]:
# age_group

data = {
    'auction_age': [
        -1, 15, 25, 35, 50, 65, 105, 80, 18, 99, 0, 579
    ]
}

# Crear el DataFrame
df = pd.DataFrame(data)

df = age_group(df, 'auction_age')
print(df)

In [ ]:
# Quiero tomar las filas que sean de tipo label 1 y que me digan si esta cerca de una festividad o no con mi train_data

# Crea un dataframe solo con las filas que tienen Label 1 a partir de train_data

df = train_data[train_data['Label'] == 1]

# Aplica la función time_features_extension al dataframe

df = time_features_extension(df)

# Imprimir las frecuencias de los valores que toman las columnas que se crearon con time_features_extension (week_day time_of_month moment_of_the_day  close_to_festivity)

for column in ['week_day', 'time_of_month', 'moment_of_the_day']:
    print(f"Columna: {column}")
    print(f"Valores únicos y su frecuencia:")
    print(df[column].value_counts())
    print("-" * 50)
 

In [ ]:
# Aplica la función time_features_extension al dataframe

test = time_features_extension(test)

# Imprimir las frecuencias de los valores que toman las columnas que se crearon con time_features_extension (week_day time_of_month moment_of_the_day  close_to_festivity)

for column in ['week_day', 'time_of_month', 'moment_of_the_day']:
    print(f"Columna: {column}")
    print(f"Valores únicos y su frecuencia:")
    print(test[column].value_counts())
    print("-" * 50)

#### List features

In [ ]:
list_features = ['auction_list_0', 'action_list_1', 'action_list_2', 'action_list_0']

for column in train_data.columns:
    if column in list_features:
        print(f"Columna: {column}")
        print(f"Valores únicos y su frecuencia:")
        print(train_data[column].value_counts())
        print("-" * 50)

In [ ]:
# Bucle para contar los NaNs en cada columna
for column in list_features:
    num_nans = train_data[column].isna().sum()
    print(f"Columna {column} tiene {num_nans} valores NaN.")

##### Funcion para hacer columnas de listas

In [ ]:
# Crear un DataFrame de prueba más pequeño
data = {
    'auction_list_0': ['["IAB3","utilities", "IAB22-2"]', '["IAB19","IAB4-5", "IAB8-9"]', None, '["IAB3","utilities", "IAB22-2"]'],
    'action_list_1': ['[-6779]', '[-6824, -6823]', None, '[-6824, -6823]' ],
    'action_list_2': ['[6871, -6543, -6544]', '[-2560, -5902]', '[-6779]', None],
    'action_list_0': ['IAB8-9', 'IAB22-2', 'IAB9-30', 'IAB9-30']
}

# Crear el DataFrame
df = pd.DataFrame(data)

print('Antes de utilizar la función expand_list_dummies_cython:')
print(df)

# Aplicar la función expand_list_dummies_cython en 'auction_list_0', 'action_list_1' y 'action_list_2'
# Aquí se asume que ya has definido y aplicado `expand_list_dummies_cython`
df = expand_list_dummies_cython(df, 'auction_list_0')
df = expand_list_dummies_cython(df, 'action_list_1')
df = expand_list_dummies_cython(df, 'action_list_2')

print('Antes de utilizar la función expand_action_list_0:')
print(df)

print()

print('Después de utilizar la función expand_action_list_0:')
df = expand_action_list_0(df)
print(df)

#### Pixels features

In [ ]:
pixels_features = ['creative_height', 'creative_width']

for column in train_data.columns:
    if column in pixels_features:
        print(f"Columna: {column}")
        print(f"Valores únicos y su frecuencia:")
        print(train_data[column].value_counts())
        print("-" * 50)


In [ ]:
# Bucle para contar los NaNs en cada columna
for column in pixels_features:
    num_nans = train_data[column].isna().sum()
    print(f"Columna {column} tiene {num_nans} valores NaN.")

In [ ]:
# Contar las filas donde ambas columnas tienen valores NaN
nans_both_columns = train_data[pixels_features].isna().all(axis=1).sum()

print(f"El número de filas donde ambas columnas tienen valores NaN es: {nans_both_columns}")

In [ ]:
# Volver a filtrar las filas donde ambas columnas son NaN
filtered_rows = train_data[train_data['creative_height'].isna() & train_data['creative_width'].isna()]

# Calcular el porcentaje de cada valor de 'Label' en las filas donde ambas columnas tienen valores NaN
label_counts = filtered_rows['Label'].value_counts(normalize=True) * 100

# Mostrar los resultados
label_counts

In [ ]:
# Filtrar las filas donde tanto creative_height como creative_width son NaN
subset_nan = train_data[train_data['creative_height'].isna() & train_data['creative_width'].isna()]

# Recorrer todas las columnas y mostrar particularidades en las 1,304,272 filas
for column in train_data.columns:
    unique_values = subset_nan[column].nunique()
    num_nans = subset_nan[column].isna().sum()
    
    print(f"Columna: {column}")
    print(f"Valores únicos: {unique_values}")
    print(f"Cantidad de NaNs: {num_nans}")
    print("-" * 50)

In [ ]:
# Valores unicos de la variable categorica auction_categorical_2
unique_values = train_data['auction_categorical_2'].unique()

unique_values

#### Creative features


In [ ]:
columns_to_check = [
    'creative_categorical_11',
    'creative_categorical_10',
    'creative_categorical_9'
]
unique_vals = {'65dcab89', '43c867fd'}

mask = train_data[columns_to_check].apply(lambda row: set(row).issubset(unique_vals) and len(set(row)) == 1, axis=1)

train_data[mask]


No podemos eliminar las columnas pero si podemos crear dos nuevas que tomen los nombres de los dos valores unicos de estas columnas

#### Auction categorical

In [4]:
entity_id  = ["auction_categorical_0", "auction_categorical_1", "auction_categorical_7", "auction_categorical_8", "auction_categorical_9", "auction_categorical_11"]

auction_categorical_variable = ["auction_categorical_2", "auction_categorical_3", "auction_categorical_4", "auction_categorical_5", "auction_categorical_6", "auction_categorical_10", "auction_categorical_12"]

In [ ]:
for column in train_data.columns:
    if column in entity_id:
        print(f"Columna: {column}")
        print(f"Valores únicos y su frecuencia:")
        print(train_data[column].value_counts())
        print("-" * 50)

In [ ]:
for column in train_data.columns:
    if column in auction_categorical_variable:
        print(f"Columna: {column}")
        print(f"Valores únicos y su frecuencia:")
        print(train_data[column].value_counts())
        print("-" * 50)

#### Categorical variables

In [11]:
categorical_variables = [
    'action_categorical_6', 'action_categorical_7', 'auction_categorical_3', 'auction_categorical_4', 'auction_categorical_5', 'auction_categorical_6', 'auction_categorical_10', 'auction_categorical_12', 'creative_categorical_1', 'creative_categorical_12', 'creative_categorical_2', 'creative_categorical_3', 'creative_categorical_4', 'creative_categorical_6', 'creative_categorical_7', 'creative_categorical_8'
]

In [ ]:
for column in train_data.columns:
    if column in categorical_variables:
        print(f"Columna: {column}")
        print(f"Valores únicos y su frecuencia:")
        print(train_data[column].value_counts())
        print("-" * 50)

In [ ]:
unique_values = {col: set(train_data[col].dropna().unique()) for col in categorical_variables}

common_columns = {}

for i in range(len(categorical_variables)):
    for j in range(i + 1, len(categorical_variables)):
        col1 = categorical_variables[i]
        col2 = categorical_variables[j]
        
        common_values = unique_values[col1].intersection(unique_values[col2])
        
        if common_values:
            if (col1, col2) not in common_columns:
                common_columns[(col1, col2)] = common_values

for col_pair, common_vals in common_columns.items():
    col1, col2 = col_pair
    print(f"\nValores comunes entre {col1} y {col2}: {common_vals}")
    print(f"Valores únicos de {col1}: {len(unique_values[col1])}")
    print(f"Valores únicos de {col2}: {len(unique_values[col2])}")

#### Entitiy IDs

In [17]:
entity_id = ['auction_categorical_0', 'auction_categorical_1', 'auction_categorical_11', 'auction_categorical_7', 'auction_categorical_8', 'auction_categorical_9']

In [ ]:
for column in train_data.columns:
    if column in entity_id:
        print(f"Columna: {column}")
        print(f"Valores únicos y su frecuencia:")
        print(train_data[column].value_counts())
        print("-" * 50)

In [ ]:
unique_values = {col: set(train_data[col].dropna().unique()) for col in entity_id}

common_columns = {}

for i in range(len(entity_id)):
    for j in range(i + 1, len(entity_id)):
        col1 = entity_id[i]
        col2 = entity_id[j]
        
        common_values = unique_values[col1].intersection(unique_values[col2])
        
        if common_values:
            if (col1, col2) not in common_columns:
                common_columns[(col1, col2)] = common_values

for col_pair, common_vals in common_columns.items():
    col1, col2 = col_pair
    print(f"\nValores comunes entre {col1} y {col2}: {common_vals}")
    print(f"Valores únicos de {col1}: {len(unique_values[col1])}")
    print(f"Valores únicos de {col2}: {len(unique_values[col2])}")

#### Business ID

In [19]:
business_columns = ['creative_categorical_0', 'creative_categorical_5']

In [ ]:
for column in train_data.columns:
    if column in business_columns:
        print(f"Columna: {column}")
        print(f"Valores únicos y su frecuencia:")
        print(train_data[column].value_counts())
        print("-" * 50)

In [23]:
# Count how many unquie values are between business_columns, and how many are common

unique_values = {col: set(train_data[col].dropna().unique()) for col in business_columns}

common_columns = {}

for i in range(len(business_columns)):
    for j in range(i + 1, len(business_columns)):
        col1 = business_columns[i]
        col2 = business_columns[j]
        
        common_values = unique_values[col1].intersection(unique_values[col2])
        
        if common_values:
            if (col1, col2) not in common_columns:
                common_columns[(col1, col2)] = common_values

for col_pair, common_vals in common_columns.items():
    col1, col2 = col_pair
    print(f"\nValores comunes entre {col1} y {col2}: {common_vals}")
    print(f"Valores únicos de {col1}: {len(unique_values[col1])}")
    print(f"Valores únicos de {col2}: {len(unique_values[col2])}")

# Entrenamiento de modelos

In [4]:
train_data_combined, _ = train_test_split(
    train_data, 
    train_size=200000, 
    random_state=random_state, 
    stratify=train_data['Label']
)

# Imprimir la cantidad de filas del dataset combinado
print(f"Cantidad de filas en el dataset combinado: {train_data_combined.shape[0]}")
print(f"Cantidad de columnas en el dataset combinado: {train_data_combined.shape[1]}")

# Ver porcentaje de clics vs no clics en la columna Label
label_counts = train_data_combined['Label'].value_counts(normalize=True) * 100
print("\nPorcentaje de clics (1) y no clics (0):")
print(label_counts)

# Cantidad de clics (1) y no clics (0)
label_counts_abs = train_data_combined['Label'].value_counts()
print("\nCantidad de clics (1) y no clics (0):")
print(label_counts_abs)

### One Hot Encoding

In [ ]:
# Dividir los datos en entrenamiento y validación con estratificación
X_train, X_val, y_train, y_val = train_test_split(
    train_data_combined.drop(columns='Label'),  # Características
    train_data_combined['Label'],               # Variable objetivo
    test_size=0.01,                              # 10% para validación
    stratify=train_data_combined['Label'],      # Estratificación basada en la variable objetivo
    random_state=random_state                   # Semilla para reproducibilidad
)

# Verificar la proporción de clases en el conjunto de entrenamiento y validación
print("Proporción en el conjunto de entrenamiento:")
print(y_train.value_counts(normalize=True))

print("Proporción en el conjunto de validación:")
print(y_val.value_counts(normalize=True))

del train_data_combined, train_data, y_val, X_val
gc.collect()

In [ ]:
X_train = process_optimized(X_train)

In [ ]:
X_train.columns.tolist()

In [ ]:
for column in X_train.columns:
    # Verifica si la suma de las frecuencias de los valores únicos es menor a 1000
    if (X_train[column] == 1).sum() < 1000:
        print(f"Columna: {column}")
        print(f"Valores únicos y su frecuencia:")
        print(X_train[column].value_counts())
        print("-" * 50)  # Separador entre columnas


In [ ]:
test_data = pd.read_csv('data/ctr_test.csv')

# X_test = process_data_with_dask(test_data, npartitions=10)

X_test = process_optimized(test_data)

# Excluir 'id' de las columnas de X_test para el reordenamiento
common_columns = [col for col in X_test.columns if col != 'id']

# Asegúrate de que las columnas en X_train coincidan con las de X_test (sin 'id')
missing_cols = set(common_columns) - set(X_train.columns)
for col in missing_cols:
    X_train[col] = 0

# Eliminar columnas extra en X_train que no están en X_test (sin contar 'id')
extra_cols = set(X_train.columns) - set(common_columns)
X_train.drop(columns=extra_cols, inplace=True)

# Reordenar X_train para que tenga el mismo orden de columnas que X_test (sin 'id')
X_train = X_train[common_columns]

# Verificar el número de columnas en X_train
print(X_train.shape[1])

In [ ]:
extra_cols

In [11]:
# Definir las columnas categóricas y numéricas
categorical_features_to_encode = X_train.select_dtypes(include=['object']).columns
numeric_features = X_train.select_dtypes(include=['number']).columns.tolist()

# Preprocesador común para imputación y codificación
common_preprocessor = ColumnTransformer(
    transformers=[
        # Imputar valores numéricos con la media
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        
        # Imputar valores categóricos con 'Desconocido' y aplicar One-Hot Encoding
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='Desconocido')),
            ('onehot', OneHotEncoder(drop='first', sparse_output=True, handle_unknown='ignore'))
        ]), categorical_features_to_encode)
    ],
    remainder='drop'  # Excluir columnas no especificadas
)

# Definir el modelo XGBoost
model_xgb = XGBClassifier(
    random_state=random_state, 
    use_label_encoder=False, 
    eval_metric='auc'
)

# Pipeline para XGBoost (mantiene matrices dispersas)
pipeline_xgb = Pipeline(steps=[
    ('preprocessor', common_preprocessor),
    ('classifier', model_xgb),
])

model_ada = AdaBoostClassifier(
    random_state=random_state
)

# Pipeline para AdaBoost
pipeline_ada = Pipeline(steps=[
    ('preprocessor', common_preprocessor),
    ('classifier', model_ada),
])

#### Hyperot XGBoost

In [12]:
# Definir el espacio de búsqueda para los hiperparámetros de XGBoost
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)), 
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]), 
    'gamma': hp.uniform('gamma', 0, 0.5),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0.5, 2),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),  # L1 regularización
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),  # L2 regularización
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),  # Muestreo a nivel de split
    'colsample_bynode': hp.uniform('colsample_bynode', 0.5, 1),  # Muestreo por nodo
    'grow_policy': hp.choice('grow_policy', ['depthwise', 'lossguide']),  # Estrategia de crecimiento
    'tree_method': hp.choice('tree_method', ['auto', 'approx', 'hist'])  # Métodos de construcción del árbol
}

# Función objetivo para Hyperopt utilizando un conjunto de validación fijo
def objective_xgb(params):
    
    # Asegurar que los parámetros sean del tipo correcto
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params['min_child_weight'] = int(params['min_child_weight'])
    
    # Definir el modelo con los hiperparámetros actuales
    model_xgb = XGBClassifier(
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        n_estimators=params['n_estimators'],
        gamma=params['gamma'],
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        scale_pos_weight=params['scale_pos_weight'],
        reg_alpha=params['reg_alpha'],  # L1 regularización
        reg_lambda=params['reg_lambda'],  # L2 regularización
        colsample_bylevel=params['colsample_bylevel'],  # Muestreo a nivel de split
        colsample_bynode=params['colsample_bynode'],  # Muestreo por nodo
        grow_policy=params['grow_policy'],  # Estrategia de crecimiento
        tree_method=params['tree_method'],  # Método de construcción del árbol
        use_label_encoder=False,  # Evitar advertencias en versiones más recientes de XGBoost
        eval_metric='auc',
        random_state=random_state
    )
    
    # Pipeline para XGBoost (mantiene matrices dispersas)
    pipeline_xgb = Pipeline(steps=[
        ('preprocessor', common_preprocessor),
        ('classifier', model_xgb),
    ])
    
    # Entrenar el modelo en el conjunto de entrenamiento
    pipeline_xgb.fit(X_train, y_train)
    
    # Predecir las probabilidades en el conjunto de validación
    y_pred_proba = pipeline_xgb.predict_proba(X_val)[:, 1]
    
    # Calcular el AUC en el conjunto de validación
    auc = roc_auc_score(y_val, y_pred_proba)
    
    # Opcional: imprimir los hiperparámetros y el AUC actual
    print(f"Hiperparámetros: {params}, AUC: {auc:.4f}")
    
    # Retornar el valor a minimizar (1 - AUC)
    return {'loss': 1 - auc, 'status': STATUS_OK}

In [ ]:
# Ejecutar la optimización
trials = Trials()

best_xgb = fmin(
    fn=objective_xgb,
    space=space,
    algo=tpe.suggest,
    max_evals=1,
    trials=trials,
    rstate=np.random.default_rng(random_state)  # Asegurar reproducibilidad
)

# No es necesario volver a mapear los hiperparámetros aquí, ya se hizo dentro de la función objetivo
print("Mejores hiperparámetros para XGBoost:")
print(best_xgb)

##### PRUEBA CON TEST

In [ ]:
# Indices
tree_method_options = ['auto', 'exact', 'approx', 'hist']
grow_policy_options = ['depthwise', 'lossguide']
n_estimators_options = [100, 200, 300, 400, 500]

# Reconstruir el modelo de XGBoost con los mejores hiperparámetros
best_model_xgb = xgb.XGBClassifier(
    n_estimators=n_estimators_options[best_xgb['n_estimators']],
    max_depth=best_xgb['max_depth'],
    learning_rate=best_xgb['learning_rate'],
    subsample=best_xgb['subsample'],
    colsample_bytree=best_xgb['colsample_bytree'],
    min_child_weight=best_xgb['min_child_weight'],
    gamma=best_xgb['gamma'],
    scale_pos_weight=best_xgb['scale_pos_weight'],
    reg_alpha=best_xgb['reg_alpha'],  # L1 regularización
    reg_lambda=best_xgb['reg_lambda'],  # L2 regularización
    colsample_bylevel=best_xgb['colsample_bylevel'],  # Muestreo a nivel de split
    colsample_bynode=best_xgb['colsample_bynode'],  # Muestreo por nodo
    grow_policy=grow_policy_options[best_xgb['grow_policy']],  # Estrategia de crecimiento
    tree_method=tree_method_options[best_xgb['tree_method']],  # Método de construcción del árbol
    random_state=random_state,
    use_label_encoder=False,  # Evitar advertencias en versiones más recientes de XGBoost
    eval_metric="auc"  # Establecer AUC como métrica de evaluación
)

# Crear un nuevo pipeline reutilizando el preprocesador original y el mejor modelo
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', common_preprocessor),
    ('classifier', model_xgb),
])

# Entrenar el modelo con los datos de entrenamiento
xgb_pipeline.fit(X_train, y_train)

In [ ]:
# Predecir en el conjunto de testeo
y_preds_xgb_test = xgb_pipeline.predict_proba(X_test)[:, 1]

# Crear el archivo de envío
submission_df_xgb = pd.DataFrame({"id": test_data["id"], "Label": y_preds_xgb_test})
submission_df_xgb["id"] = submission_df_xgb["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros
file_name_xgb = (
    f"xgboost_preds_ne_{best_xgb['n_estimators']}_"
    f"md_{handle_none(best_xgb['max_depth'])}_"
    f"lr_{round(best_xgb['learning_rate'], 2)}_"
    f"ss_{round(best_xgb['subsample'], 2)}_"
    f"csb_{round(best_xgb['colsample_bytree'], 2)}_"
    f"cb_level_{round(best_xgb['colsample_bylevel'], 2)}_"
    f"cb_node_{round(best_xgb['colsample_bynode'], 2)}_"
    f"mcw_{round(best_xgb['min_child_weight'], 2)}_"
    f"gamma_{round(best_xgb['gamma'], 2)}_"
    f"ra_{round(best_xgb['reg_alpha'], 2)}_"
    f"rl_{round(best_xgb['reg_lambda'], 2)}_"
    f"spw_{round(best_xgb['scale_pos_weight'], 2)}_"
    f"gp_{grow_policy_options[best_xgb['grow_policy']]}_"
    f"tm_{tree_method_options[best_xgb['tree_method']]}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_xgb.to_csv(os.path.join("submits", file_name_xgb), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_xgb}")

#### Hyperot AdaBoost

In [ ]:
space_ada = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 10),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(1.0)),
    'algorithm': hp.choice('algorithm', ['SAMME', 'SAMME.R']),
    
    # Hiperparámetros del estimador base (DecisionTreeClassifier)
    'base_max_depth': hp.quniform('base_max_depth', 1, 15, 1),
    'base_min_samples_split': hp.quniform('base_min_samples_split', 2, 20, 1),
    'base_min_samples_leaf': hp.quniform('base_min_samples_leaf', 1, 20, 1),
    'base_max_features': hp.choice('base_max_features', ['auto', 'sqrt', 'log2', None])
}

def objective_ada(params):
    # Convertir hiperparámetros a enteros si es necesario
    n_estimators = int(params['n_estimators'])
    base_max_depth = int(params['base_max_depth'])
    base_min_samples_split = int(params['base_min_samples_split'])
    base_min_samples_leaf = int(params['base_min_samples_leaf'])
    
    # Seleccionar el algoritmo
    algorithm = params['algorithm']
    
    # Manejar 'base_max_features' que puede ser None
    base_max_features = params['base_max_features']
    if base_max_features == 'auto':
        base_max_features = 'sqrt'
    elif base_max_features == 'log2':
        base_max_features = 'log2'
    elif base_max_features is None:
        base_max_features = None
    else:
        base_max_features = base_max_features  # 'sqrt' ya está manejado
    
    # Definir el estimador base
    base_estimator = DecisionTreeClassifier(
        max_depth=base_max_depth,
        min_samples_split=base_min_samples_split,
        min_samples_leaf=base_min_samples_leaf,
        max_features=base_max_features,
        random_state=random_state
    )
    
    # Definir el clasificador AdaBoost
    clf = AdaBoostClassifier(
        base_estimator=base_estimator,
        n_estimators=n_estimators,
        learning_rate=params['learning_rate'],
        algorithm=algorithm,
        random_state=random_state
    )
    
    # Definir el pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', common_preprocessor),
        ('classifier', clf),
    ])
    
    # Definir la validación cruzada
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    
    # Evaluar el modelo usando cross_val_score
    auc = cross_val_score(pipeline, X_train, y_train, cv=skf, scoring='roc_auc').mean()
    
    # Opcional: imprimir los hiperparámetros y el AUC actual
    print(f"Hiperparámetros: {params}, AUC: {auc:.4f}")
    
    # Retornar el valor a minimizar (1 - AUC)
    return {'loss': 1 - auc, 'status': STATUS_OK}

In [ ]:
trials_ada = Trials()

# Ejecutar la optimización
best_ada = fmin(
    fn=objective_ada,
    space=space_ada,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials_ada,
    rstate=np.random.RandomState(random_state)  # Asegurar reproducibilidad
)

# Procesar los hiperparámetros seleccionados
best_ada['n_estimators'] = int(best_ada['n_estimators'])
best_ada['base_max_depth'] = int(best_ada['base_max_depth'])
best_ada['base_min_samples_split'] = int(best_ada['base_min_samples_split'])
best_ada['base_min_samples_leaf'] = int(best_ada['base_min_samples_leaf'])
best_ada['algorithm'] = ['SAMME', 'SAMME.R'][best_ada['algorithm']]
best_ada['base_max_features'] = ['auto', 'sqrt', 'log2', None][best_ada['base_max_features']]

print("Mejores hiperparámetros para AdaBoost:")
print(best_ada)

In [ ]:
# Reconstruir el modelo de AdaBoost con los mejores hiperparámetros
best_base_estimator_ada = DecisionTreeClassifier(
    max_depth=best_ada['base_max_depth'],
    min_samples_split=best_ada['base_min_samples_split'],
    min_samples_leaf=best_ada['base_min_samples_leaf'],
    max_features=best_ada['base_max_features'],
    random_state=random_state
)

best_clf_ada = AdaBoostClassifier(
    base_estimator=best_base_estimator_ada,
    n_estimators=best_ada['n_estimators'],
    learning_rate=best_ada['learning_rate'],
    algorithm=best_ada['algorithm'],
    random_state=random_state
)

# Crear un nuevo pipeline reutilizando el preprocesador original y el mejor modelo
ada_pipeline = Pipeline(steps=[
    ('preprocessor', common_preprocessor),
    ('classifier', best_clf_ada),
])

# Entrenar el modelo con los datos de entrenamiento
ada_pipeline.fit(X_train, y_train)

In [ ]:
# Predecir en el conjunto de testeo
y_preds_ada_test = ada_pipeline.predict(X_test)

# Crear el archivo de envío
submission_df_ada = pd.DataFrame({
    "id": test.index,  # Asegúrate de que 'test' tiene una columna 'id'; ajusta si es necesario
    "Label": y_preds_ada_test
})

# Asegurar que la columna 'id' es de tipo entero
submission_df_ada["id"] = submission_df_ada["id"].astype(int)

# Función para manejar valores None en nombres de archivos
def handle_none(value):
    return 'none' if value is None else value

# Opciones para los parámetros que pueden tener opciones específicas
algorithm_options_ada = {'SAMME': 'samm', 'SAMME.R': 'sammr'}
max_features_options_ada = {'auto': 'auto', 'sqrt': 'sqrt', 'log2': 'log2', None: 'none'}

# Crear el nombre del archivo basado en los mejores hiperparámetros
file_name_ada = (
    f"adaboost_preds_ne_{best_ada['n_estimators']}_"
    f"lr_{round(best_ada['learning_rate'], 4)}_"
    f"alg_{algorithm_options_ada[best_ada['algorithm']]}_"
    f"md_{best_ada['base_max_depth']}_"
    f"mss_{best_ada['base_min_samples_split']}_"
    f"msl_{best_ada['base_min_samples_leaf']}_"
    f"mf_{max_features_options_ada[best_ada['base_max_features']]}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_ada.to_csv(os.path.join("submits", file_name_ada), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_ada}")
